In [1]:
import pandas as pd

# Load data from the Excel file
df = pd.read_excel('depot_route_time_matrix.xlsx')

# Separate the number of buses required per route (last row)
buses_required = df.iloc[-1, 1:-1]  # Exclude 'Depot' and 'Capacity' columns
buses_required.index = df.columns[1:-1]  # Set the route names as the index

# Remove the last row (buses required) from the main data frame
df = df.iloc[:-1, :]

# Extract depot capacities (last column)
depot_capacity_df = df[['Depot', 'Capacity']].set_index('Depot')

# Extract route names (all columns except 'Depot' and 'Capacity')
route_columns = [col for col in df.columns if col not in ['Depot', 'Capacity']]

# Function to allocate buses
def allocate_buses(route_list, buses_required):
    allocation = []

    for route in route_list:
        # Get the number of buses required for this route
        buses_per_route = buses_required[route]

        # Get the depots sorted by time for this route
        depots_for_route = df[['Depot', route, 'Capacity']].sort_values(route)

        buses_allocated = 0
        for index, depot in depots_for_route.iterrows():
            depot_name = depot['Depot']
            available_capacity = depot_capacity_df.loc[depot_name, 'Capacity']

            if available_capacity > 0:
                buses_to_allocate = min(available_capacity, buses_per_route - buses_allocated)

                if buses_to_allocate > 0:
                    # Update depot capacity
                    depot_capacity_df.loc[depot_name, 'Capacity'] -= buses_to_allocate

                    # Record allocation
                    allocation.append({'Route': route, 'Depot': depot_name, 'Buses Allocated': buses_to_allocate})
                    buses_allocated += buses_to_allocate

                # Stop if we have allocated all buses for the route
                if buses_allocated == buses_per_route:
                    break

        if buses_allocated < buses_per_route:
            print(f"Warning: Could not allocate all buses for route {route}")

    return pd.DataFrame(allocation)

# Example usage
route_list = route_columns  # Get the list of routes
allocation_df = allocate_buses(route_list, buses_required)

# Save the result to a new Excel file
allocation_df.to_excel('Depot_Route_Bus_Allocation_Results.xlsx', index=False)
# print("Bus allocation completed and saved to 'bus_allocation_results.xlsx'")
